In [2]:
import os
import numpy as np
from scipy.io import mmread, mmwrite
from scipy.sparse import isspmatrix
from scipy.sparse import csr_matrix

def process_matrices(data_directory):
    # Create subdirectories if they do not exist
    matrices_dir = os.path.join(data_directory, 'matrices')
    b_vectors_dir = os.path.join(data_directory, 'b_vectors')
    os.makedirs(matrices_dir, exist_ok=True)
    os.makedirs(b_vectors_dir, exist_ok=True)

    # Traverse all nested directories
    for root, dirs, files in os.walk(data_directory):
        for dir_name in dirs:
            matrix_dir = os.path.join(root, dir_name)
            filename_root = dir_name

            # Paths to the matrix file and b vector file
            matrix_file = os.path.join(matrix_dir, f"{filename_root}.mtx")
            b_vector_file = os.path.join(b_vectors_dir, f"{filename_root}_b.mtx")

            if os.path.exists(matrix_file):
                print(f"Processing matrix: {matrix_file}")

                # Load the matrix
                matrix = mmread(matrix_file)

                # Ensure the matrix is in sparse format
                if not isspmatrix(matrix):
                    matrix = csr_matrix(matrix)

                # Check if the _b.mtx file already exists
                if not os.path.exists(b_vector_file):
                    print(f"{b_vector_file} not found. Generating solution vector...")

                    # Generate a random x vector
                    x_vector = np.random.rand(matrix.shape[1])

                    # Multiply matrix by x to get the solution vector b
                    b_vector = matrix.dot(x_vector)

                    # Save the result in Market Matrix format in the b_vectors subdirectory
                    mmwrite(b_vector_file, csr_matrix(b_vector.reshape(-1, 1)))
                    print(f"Saved b vector: {b_vector_file}")
                else:
                    print(f"{b_vector_file} already exists, skipping.")

                # Move the original matrix to the matrices subdirectory
                new_matrix_path = os.path.join(matrices_dir, f"{filename_root}.mtx")
                os.rename(matrix_file, new_matrix_path)
                print(f"Moved matrix file to {new_matrix_path}")
            else:
                print(f"{matrix_file} not found, skipping.")

if __name__ == "__main__":
    # Set the directory containing the data (replace with your directory path)
    data_directory = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices"
    
    process_matrices(data_directory)


/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/mark3jac020sc/mark3jac020sc.mtx not found, skipping.
/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/cage10/cage10.mtx not found, skipping.
/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/stokes64/stokes64.mtx not found, skipping.
/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/TSOPF_FS_b9_c6/TSOPF_FS_b9_c6.mtx not found, skipping.
/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/har_10NN/har_10NN.mtx not found, skipping.
/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/JapaneseVowelsSmall_10NN/JapaneseVowelsSmall_10NN.mtx not found, skipping.
/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/TSC_OPF_300/TSC_OPF_300.mtx not found, skipping.
/Users/tonylizza/Documents/Machine

In [4]:
import os
import shutil

def move_b_vectors(data_directory):
    # Create the b_vectors subdirectory if it doesn't exist
    b_vectors_dir = os.path.join(data_directory, 'b_vectors')
    os.makedirs(b_vectors_dir, exist_ok=True)

    # Traverse all nested directories
    for root, dirs, files in os.walk(data_directory):
        for dir_name in dirs:
            matrix_dir = os.path.join(root, dir_name)
            filename_root = dir_name

            # Path to the existing _b.mtx file
            b_vector_file = os.path.join(matrix_dir, f"{filename_root}_b.mtx")

            if os.path.exists(b_vector_file):
                print(f"Found b vector: {b_vector_file}")

                # Destination path for the b vector
                destination_file = os.path.join(b_vectors_dir, f"{filename_root}_b.mtx")

                # Move the b vector to the b_vectors subdirectory
                shutil.move(b_vector_file, destination_file)
                print(f"Moved b vector to {destination_file}")
            else:
                print(f"No b vector found for {filename_root}, skipping.")

if __name__ == "__main__":
    # Set the directory containing the data (replace with your directory path)
    data_directory = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU"
    
    move_b_vectors(data_directory)


No b vector found for mark3jac020sc, skipping.
No b vector found for cage10, skipping.
No b vector found for stokes64, skipping.
No b vector found for TSOPF_FS_b9_c6, skipping.
No b vector found for har_10NN, skipping.
No b vector found for JapaneseVowelsSmall_10NN, skipping.
No b vector found for TSC_OPF_300, skipping.
No b vector found for b_vectors, skipping.
No b vector found for graham1, skipping.
No b vector found for nemeth17, skipping.
No b vector found for nemeth10, skipping.
No b vector found for ted_A_unscaled, skipping.
No b vector found for Reuters911, skipping.
No b vector found for nemeth26, skipping.
No b vector found for pesa, skipping.
No b vector found for c-46, skipping.
No b vector found for TSOPF_FS_b162_c1, skipping.
No b vector found for nemeth19, skipping.
No b vector found for nemeth21, skipping.
No b vector found for flowmeter5, skipping.
No b vector found for c-41, skipping.
No b vector found for big, skipping.
No b vector found for hangGlider_3, skipping.
N

In [5]:
import os
import numpy as np
from scipy.io import mmread, mmwrite
from scipy.sparse import isspmatrix
from scipy.sparse.linalg import spilu
from scipy.sparse import csr_matrix

def create_ilu_preconditioner(matrix_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop through all .mtx files in the matrix directory
    for file_name in os.listdir(matrix_dir):
        if file_name.endswith(".mtx") and not file_name.endswith("_ilu.mtx"):
            matrix_path = os.path.join(matrix_dir, file_name)
            print(f"Processing matrix: {matrix_path}")

            # Load the matrix from the .mtx file
            matrix = mmread(matrix_path)

            # Ensure the matrix is in sparse format
            if not isspmatrix(matrix):
                matrix = csr_matrix(matrix)

            try:
                # Create ILU preconditioner (ILU decomposition)
                ilu = spilu(matrix)

                # Convert the ILU preconditioner to a sparse matrix
                ilu_matrix = ilu.L + ilu.U - csr_matrix(np.eye(matrix.shape[0]))

                # Define the output filename and path
                output_filename = file_name.replace(".mtx", "_ilu.mtx")
                output_path = os.path.join(output_dir, output_filename)

                # Save the ILU preconditioner as a .mtx file
                mmwrite(output_path, ilu_matrix)
                print(f"Saved ILU preconditioner to: {output_path}")

            except Exception as e:
                print(f"Failed to create ILU for {file_name}: {e}")

if __name__ == "__main__":
    # Set the directory containing the matrix files
    matrix_directory = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/more_matrices"  # Replace with your directory
    output_directory = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/ilu_matrices"  # Directory to save ILU matrices

    # Process matrices and create ILU preconditioners
    create_ilu_preconditioner(matrix_directory, output_directory)


Processing matrix: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/kmnist_norm_10NN.mtx


/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:437: SparseEfficiencyWarning: spilu converted its input to CSC format
  warn('spilu converted its input to CSC format',


Failed to create ILU for kmnist_norm_10NN.mtx: Factor is exactly singular
Processing matrix: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/lowThrust_4.mtx
Failed to create ILU for lowThrust_4.mtx: Factor is exactly singular
Processing matrix: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/nemeth13.mtx
Saved ILU preconditioner to: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/ilu_matrices/nemeth13_ilu.mtx
Processing matrix: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/nemeth07.mtx
Saved ILU preconditioner to: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/ilu_matrices/nemeth07_ilu.mtx
Processing matrix: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/nemeth06.mtx
Saved ILU preconditioner to: /Users/tonylizza/Docu

In [6]:
import os

def get_root_name(filename, suffix):
    """
    Extract the root name from a filename by removing the suffix (e.g., '_ilu.mtx', '_b.mtx').
    """
    return filename.replace(suffix, '')

def get_files_with_suffix(directory, suffix):
    """
    Get a set of root names from filenames in a directory, removing the specified suffix.
    """
    return {get_root_name(f, suffix) for f in os.listdir(directory) if f.endswith(suffix)}

def remove_files_by_root_names(directory, root_names, suffix):
    """
    Remove files in the given directory that have a root name not in the specified root_names set.
    """
    for f in os.listdir(directory):
        if f.endswith(suffix):
            root_name = get_root_name(f, suffix)
            if root_name not in root_names:
                file_path = os.path.join(directory, f)
                print(f"Removing file: {file_path}")
                os.remove(file_path)

def clean_directories(matrices_dir, ilu_matrices_dir, b_vectors_dir):
    # Get the set of root names in the ilu_matrices directory (those that have ILU preconditioners)
    ilu_root_names = get_files_with_suffix(ilu_matrices_dir, '_ilu.mtx')

    # Remove files from the matrices directory that don't have corresponding ILU files
    remove_files_by_root_names(matrices_dir, ilu_root_names, '.mtx')

    # Remove files from the b_vectors directory that don't have corresponding ILU files
    remove_files_by_root_names(b_vectors_dir, ilu_root_names, '_b.mtx')

if __name__ == "__main__":
    # Specify the directories
    matrices_dir = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices"
    ilu_matrices_dir = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/ilu_matrices"
    b_vectors_dir = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/b_vectors"

    # Clean up the directories by removing unmatched files
    clean_directories(matrices_dir, ilu_matrices_dir, b_vectors_dir)


Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/kmnist_norm_10NN.mtx
Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/lowThrust_4.mtx
Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/wiki-RfA.mtx
Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/ted_A_unscaled.mtx
Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/Reuters911.mtx
Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/cz10228.mtx
Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/sinc15.mtx
Removing file: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/large_ILU/matrices/mnist

In [7]:
import os
import numpy as np
from scipy.io import mmread, mmwrite
from scipy.sparse import isspmatrix
from scipy.sparse.linalg import spilu
from scipy.sparse import csr_matrix
import shutil

def process_matrices_and_create_ilu(data_directory):
    # Create subdirectories if they do not exist
    matrices_dir = os.path.join(data_directory, 'matrices')
    b_vectors_dir = os.path.join(data_directory, 'b_vectors')
    ilu_matrices_dir = os.path.join(data_directory, 'ilu_matrices')
    
    os.makedirs(matrices_dir, exist_ok=True)
    os.makedirs(b_vectors_dir, exist_ok=True)
    os.makedirs(ilu_matrices_dir, exist_ok=True)

    # Traverse all nested directories
    for root, dirs, files in os.walk(data_directory):
        for dir_name in dirs:
            matrix_dir = os.path.join(root, dir_name)
            filename_root = dir_name

            # Paths to the matrix file and b vector file
            matrix_file = os.path.join(matrix_dir, f"{filename_root}.mtx")
            b_vector_file = os.path.join(b_vectors_dir, f"{filename_root}_b.mtx")
            ilu_output_file = os.path.join(ilu_matrices_dir, f"{filename_root}_ilu.mtx")

            if os.path.exists(matrix_file):
                print(f"Processing matrix: {matrix_file}")

                # Load the matrix
                matrix = mmread(matrix_file)

                # Ensure the matrix is in sparse format
                if not isspmatrix(matrix):
                    matrix = csr_matrix(matrix)

                try:
                    # Create ILU preconditioner (ILU decomposition)
                    print(f"Attempting to create ILU preconditioner for {matrix_file}...")
                    ilu = spilu(matrix)

                    # Convert the ILU preconditioner to a sparse matrix
                    ilu_matrix = ilu.L + ilu.U - csr_matrix(np.eye(matrix.shape[0]))

                    # Save the ILU preconditioner as a .mtx file
                    mmwrite(ilu_output_file, ilu_matrix)
                    print(f"Saved ILU preconditioner to: {ilu_output_file}")

                    # Move the original matrix file to the matrices subdirectory
                    new_matrix_path = os.path.join(matrices_dir, f"{filename_root}.mtx")
                    shutil.move(matrix_file, new_matrix_path)
                    print(f"Moved matrix file to {new_matrix_path}")

                    # Check if the _b.mtx file already exists
                    if os.path.exists(b_vector_file):
                        print(f"{b_vector_file} already exists, moving to b_vectors directory.")
                    else:
                        # Generate a random x vector
                        print(f"{b_vector_file} not found. Generating solution vector...")
                        x_vector = np.random.rand(matrix.shape[1])

                        # Multiply matrix by x to get the solution vector b
                        b_vector = matrix.dot(x_vector)

                        # Save the result in Market Matrix format in the b_vectors subdirectory
                        mmwrite(b_vector_file, csr_matrix(b_vector.reshape(-1, 1)))
                        print(f"Saved b vector: {b_vector_file}")

                except Exception as e:
                    print(f"Failed to create ILU for {filename_root}: {e}")
                    print("Skipping move of matrix and b vector files due to ILU failure.")
                    continue  # Skip moving files if ILU fails

if __name__ == "__main__":
    # Set the directory containing the data
    data_directory = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/new_matrices"
    
    # Process matrices, create ILU preconditioners, and move files
    process_matrices_and_create_ilu(data_directory)


Processing matrix: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/new_matrices/bcspwr01/bcspwr01.mtx
Attempting to create ILU preconditioner for /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/new_matrices/bcspwr01/bcspwr01.mtx...
Saved ILU preconditioner to: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/new_matrices/ilu_matrices/bcspwr01_ilu.mtx
Moved matrix file to /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/new_matrices/matrices/bcspwr01.mtx
/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/new_matrices/b_vectors/bcspwr01_b.mtx not found. Generating solution vector...
Saved b vector: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/new_matrices/b_vectors/bcspwr01_b.mtx


In [3]:
import os
import numpy as np
from scipy.io import mmread, mmwrite
from scipy.sparse import csr_matrix

def create_b_vector(matrix_file):
    # Load the matrix from the .mtx file
    matrix = mmread(matrix_file)
    
    # Ensure the matrix is in sparse format
    if not isinstance(matrix, csr_matrix):
        matrix = csr_matrix(matrix)

    # Create a random x vector
    x_vector = np.random.rand(matrix.shape[1])

    # Multiply matrix by x to get the b vector
    b_vector = matrix.dot(x_vector)

    # Create the output filename for the b vector
    filename_root = os.path.splitext(matrix_file)[0]
    b_vector_file = f"{filename_root}_b.mtx"

    # Save the b vector as an .mtx file
    mmwrite(b_vector_file, csr_matrix(b_vector.reshape(-1, 1)))
    print(f"Saved b vector to: {b_vector_file}")

if __name__ == "__main__":
    # Example usage
    matrix_file = "/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/matrices/Trefethen_20.mtx"
    create_b_vector(matrix_file)


Saved b vector to: /Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/matrices/Trefethen_20_b.mtx
